In [78]:
import time
import pandas as pd
import gzip
# 0 - Revoke, 1 - Post, 2 - Match

In [79]:
# CHANGE THIS
WORKING_DIR = r"D:\DataMining\MOEX-FX\MOEX-FX\2018-03\\"
OrderLog = "OrderLog20180301"
with gzip.open(WORKING_DIR+'OrderLog20180301.txt.gz', 'rb') as f:
    order_log = pd.read_csv(f)

with gzip.open(WORKING_DIR+'TradeLog20180301.txt.gz', 'rb') as f:
    trade_log = pd.read_csv(f)

In [80]:
class Action:
    REVOKE = 0
    POST = 1
    MATCH = 2

In [81]:
CODES = ['USD000000TOD', 'USD000UTSTOM', 'EUR_RUB__TOD', 'EUR_RUB__TOM', 'EURUSD000TOD', 'EURUSD000TOM']

In [82]:
instruments_info = {'USD000000TOD': {'SCHEDULE': 174500000000, 'PRICE_STEP': 0.0025, 'INDEX':0},
                    'USD000UTSTOM': {'SCHEDULE': 235000000000, 'PRICE_STEP': 0.0025, 'INDEX':1},
                    'EUR_RUB__TOD': {'SCHEDULE': 150000000000, 'PRICE_STEP': 0.0025, 'INDEX':2},
                    'EUR_RUB__TOM': {'SCHEDULE': 235000000000, 'PRICE_STEP': 0.0025, 'INDEX':3},
                    'EURUSD000TOM': {'SCHEDULE': 235000000000, 'PRICE_STEP': 0.00001, 'INDEX':4},
                    'EURUSD000TOD': {'SCHEDULE': 150000000000, 'PRICE_STEP': 0.00001, 'INDEX':5}}

In [83]:
# filtering orders
df_order = order_log[(order_log['SECCODE'].isin(CODES)) & (order_log['TIME'] < 235000000000)].copy()[:1000]
df_trade = trade_log[trade_log['SECCODE'].isin(CODES)].copy()

# harvard professor wrote this +100000 IQ code
# to sort actions by the following order: post, match, revoke
prep_dic = {Action.POST: 0, Action.MATCH: 1, Action.REVOKE: 2}
unprep_dic = { v: k for k, v in prep_dic.items() } # Inverse `prep_dic`

df_order['ACTION'] = df_order['ACTION'].apply(lambda num: prep_dic[num])
df_order.sort_values(by = ['TIME', 'ACTION'], inplace=True)
df_order['ACTION'] = df_order['ACTION'].apply(lambda num: unprep_dic[num])

In [84]:
df_order.head()

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,EUR_RUB__TOD,B,100000050299,1,1,68.610,1000000,NaN,NaN
1,2,USD000000TOD,S,100000050316,2,1,56.600,1000000,NaN,NaN
2,3,EUR_RUB__TOD,S,100000050325,3,1,69.065,1000000,NaN,NaN
3,4,EUR_RUB__TOM,B,100000050353,4,1,68.725,1000000,NaN,NaN
4,5,USD000000TOD,B,100000050361,5,1,56.210,1000000,NaN,NaN


In [85]:
df_order[df_order['NO'] == 100]

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
99,100,EUR_RUB__TOM,S,100000210798,57,0,68.875,1000000,NaN,NaN


In [86]:
df_trade.head()

,TRADENO,SECCODE,TIME,BUYORDERNO,SELLORDERNO,PRICE,VOLUME
0,163897149,USD000000TOD,100000,88,90,56.3800,70000
1,163897150,USD000UTSTOM,100001,296,628,56.4000,1000
2,163897151,USD000UTSTOM,100001,296,663,56.4000,1000
3,163897152,USD000UTSTOM,100001,701,184,56.4475,1000
4,163897153,USD000UTSTOM,100001,296,881,56.4000,118000


In [87]:
df_trade.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59713 entries, 0 to 61879
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   TRADENO      59713 non-null  int64  
 1   SECCODE      59713 non-null  object 
 2   TIME         59713 non-null  int64  
 3   BUYORDERNO   59713 non-null  int64  
 4   SELLORDERNO  59713 non-null  int64  
 5   PRICE        59713 non-null  float64
 6   VOLUME       59713 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 3.6+ MB


In [88]:
df_order['TRADENO'].value_counts()

163897154.0    2
163897153.0    2
163897151.0    2
163897152.0    2
163897149.0    2
163897150.0    2
Name: TRADENO, dtype: int64

In [89]:
# Helper functions

def drop_by_entry(df_cur, entry, inplace=True):
    df_cur.drop(df_cur[df_cur['ORDERNO'] == entry['ORDERNO']].index, inplace=inplace)

def reduce_by_entry(df_cur, entry, FROM, TO):
    df_cur.loc[df_cur[df_cur['ORDERNO'] == entry['ORDERNO']].index, ['VOLUME']] = FROM['VOLUME'] - TO['VOLUME']

In [90]:
total_mistakes = 0

def print_error(error):
    global total_mistakes
    print('-' * 40)
    print(f'in row: {row["NO"]}')
    print(error)
    print('-' * 40)
    print()
    total_mistakes += 1

def check_exists(df_cur, row, NO='ORDERNO'):
    # check if the given ORDERNO exists
    # sub_df = df_cur[df_cur[NO] == row[NO]]
    df_cur_searchable = df_cur.set_index(NO)
    # sub_df = df_cur[df_cur_searchable.isin(row[NO])]
    sub_df = df_cur[df_cur_searchable==row[NO]]
    if len(sub_df) > 0:
        return True, sub_df
    # printing the error
    else:
        print_error(f"ERROR: Record with {NO} {row[NO]} doesn't exist")
        return False, None


def handle_revoke(df_cur, row):
    # check if the given ORDERNO exists
    check, sub_df = check_exists(df_cur, row)
    df_cur_searchable = df_cur.set_index('ORDERNO')
    if check:
        series = sub_df.iloc[0]
        # check if revoking volume is not greater than the current one
        if series['VOLUME'] >= row['VOLUME']:
            if series['VOLUME'] == row['VOLUME']:
                # removing the record
                # df_cur.drop(df_cur[df_cur['ORDERNO'] == row['ORDERNO']].index, inplace=True)
                # df_cur.drop(df_cur_searchable.isin(row['ORDERNO']).index, inplace=True)
                df_cur.drop(df_cur[df_cur_searchable==row['ORDERNO']].index, inplace=True)

            else:
                # reducing the amount
                # df_cur.loc[df_cur[df_cur['ORDERNO'] == row['ORDERNO']].index, ['VOLUME']] = series['VOLUME'] - row['VOLUME']
                # df_cur.iat[df_cur_searchable.isin(row['ORDERNO']).index, 'VOLUME'] = series['VOLUME'] - row['VOLUME']
                df_cur.iat[df_cur[df_cur_searchable==row['ORDERNO']].index, 'VOLUME'] = series['VOLUME'] - row['VOLUME']
        # printing the error
        else:
            # Delete negative number
            # df_cur.drop(df_cur[df_cur['ORDERNO'] == row['ORDERNO']].index, inplace=True)
            # df_cur.drop(df_cur_searchable.isin(row['ORDERNO']).index, inplace=True)
            df_cur.drop(df_cur[df_cur_searchable==row['ORDERNO']].index, inplace=True)
            print_error("ERROR: Cannot revoke more that there is")


def handle_match(df_cur, row):
    global df_trade
    # check if the given ORDERNO exists
    check, sub_df = check_exists(df_cur, row)
    df_cur_searchable = df_cur.set_index('ORDERNO')
    if check:
        # check if the given tradeno exists in trade logs
        check, sub_trade = check_exists(df_trade, row, 'TRADENO')
        if check:
            series_trade = sub_trade.iloc[0]

            # seller = df_cur[df_cur['ORDERNO'] == series_trade['SELLORDERNO']]
            # seller = df_cur[df_cur_searchable.isin(series_trade['SELLORDERNO'])]
            seller = df_cur[df_cur_searchable==series_trade['SELLORDERNO']]
            # buyer = df_cur[df_cur['ORDERNO'] == series_trade['BUYORDERNO']]
            # buyer = df_cur[df_cur_searchable.isin(series_trade['BUYORDERNO'])]
            buyer = df_cur[df_cur_searchable==series_trade['BUYORDERNO']]

            # check if seller and buyer exist
            if len(seller) > 0 and len(buyer) > 0:
                seller = seller.iloc[0]
                buyer = buyer.iloc[0]

                # check if the volume is ok
                if seller['VOLUME'] >= row['VOLUME'] and buyer['VOLUME'] >= row['VOLUME']:
                    if seller['VOLUME'] == row['VOLUME']:
                        # removing the record
                        # df_cur.drop(df_cur[df_cur['ORDERNO'] == seller['ORDERNO']].index, inplace=True)
                        # df_cur.drop(df_cur_searchable.isin(seller['ORDERNO']).index, inplace=True)
                        df_cur.drop(df_cur[df_cur_searchable==seller['ORDERNO']].index, inplace=True)
                    else:
                        # reducing the amount
                        # df_cur.loc[df_cur['ORDERNO'] == seller['ORDERNO'], ['VOLUME']] = seller['VOLUME'] - row['VOLUME']
                        # df_cur.iat[df_cur_searchable.isin(seller['ORDERNO']).index, 'VOLUME'] = seller['VOLUME'] - row['VOLUME']
                        df_cur.iat[df_cur[df_cur_searchable==seller['ORDERNO']].index, 'VOLUME'] = seller['VOLUME'] - row['VOLUME']

                    if buyer['VOLUME'] == row['VOLUME']:
                        # removing the record
                        # df_cur.drop(df_cur[df_cur['ORDERNO'] == buyer['ORDERNO']].index, inplace=True)
                        # df_cur.drop(df_cur_searchable.isin(buyer['ORDERNO']).index, inplace=True)
                        df_cur.drop(df_cur[df_cur_searchable==buyer['ORDERNO']].index, inplace=True)
                    else:
                        # reducing the amount
                        # df_cur.loc[df_cur['ORDERNO'] == buyer['ORDERNO'], ['VOLUME']] = buyer['VOLUME'] - row['VOLUME']
                        # df_cur.iat[df_cur_searchable.isin(buyer['ORDERNO']).index, 'VOLUME'] = seller['VOLUME'] - row['VOLUME']
                        df_cur.iat[df_cur[df_cur_searchable==buyer['ORDERNO']].index, 'VOLUME'] = seller['VOLUME'] - row['VOLUME']

                # printing the error
                else:
                    # Delete negative numbers
                    if seller['VOLUME'] < row['VOLUME']:
                        # df_cur.drop(df_cur[df_cur['ORDERNO'] == seller['ORDERNO']].index, inplace=True)
                        # df_cur.drop(df_cur_searchable.isin(seller['ORDERNO']).index, inplace=True)
                        df_cur.drop(df_cur[df_cur_searchable==seller['ORDERNO']].index, inplace=True)
                    else:
                        # df_cur.loc[df_cur['ORDERNO'] == seller['ORDERNO'], ['VOLUME']] = seller['VOLUME'] - row['VOLUME']
                        # df_cur.iat[df_cur_searchable.isin(seller['ORDERNO']).index, 'VOLUME'] = seller['VOLUME'] - row['VOLUME']
                        df_cur.iat[df_cur[df_cur_searchable==seller['ORDERNO']].index, 'VOLUME'] = seller['VOLUME'] - row['VOLUME']

                    # Delete negative numbers
                    if buyer['VOLUME'] < row['VOLUME']:
                       # df_cur.drop(df_cur[df_cur['ORDERNO'] == buyer['ORDERNO']].index, inplace=True)
                       #  df_cur.drop(df_cur_searchable.isin(buyer['ORDERNO']).index, inplace=True)
                        df_cur.drop(df_cur[df_cur_searchable==buyer['ORDERNO']].index, inplace=True)
                    else:
                        # df_cur.loc[df_cur['ORDERNO'] == buyer['ORDERNO'], ['VOLUME']] = buyer['VOLUME'] - row['VOLUME']
                        # df_cur.iat[df_cur_searchable.isin(buyer['ORDERNO']).index, 'VOLUME'] = seller['VOLUME'] - row['VOLUME']
                        df_cur.iat[df_cur[df_cur_searchable==buyer['ORDERNO']].index, 'VOLUME'] = seller['VOLUME'] - row['VOLUME']


                    print_error("ERROR: Not enough amount for buying and/or selling")

            # printing the error
            else:
                print_error(f"ERROR: There's no such buyer and/or seller")

In [91]:
df_order[df_order['PRICE'] == 0]

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
861,862,USD000UTSTOM,B,100001279110,701,1,0.0,1000,NaN,NaN
862,863,USD000UTSTOM,B,100001279110,701,2,0.0,1000,163897152.0,56.4475


In [92]:
# ГОВНОКОД NO.1
# блин хз возможно класс больше не нужен

class Spectrum:
  def __init__(self, seccode):
        self.seccode = seccode
        self.best_ask = 100000000
        self.best_bid = -1
        self.bids = [0]*10
        self.asks = [0]*10

In [93]:
# Helper functions
import math

def distance_idx(dif, step):
    return math.floor(dif/(step*5))

In [94]:
# ГОВНОКОД NO.2

spectrums = dict()
for i in instruments_info.keys():
  spectrums[i] = Spectrum(i)

In [95]:
# ГОВНОКОД NO.3

def change_spectrums(df_cur, row):
    seccode = row['SECCODE']
    step = instruments_info[seccode]['PRICE_STEP']

    spectrum = spectrums[seccode]

    new_price = row['PRICE']

    if row['ACTION'] == Action.POST:

        if row['BUYSELL'] == 'B':
            if new_price > spectrum.best_bid:
                # temp_b = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'B')]
                temp_b = df_cur[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'B')]

                spectrum.best_bid = new_price
                new_bids = [0]*10

                for index, row in temp_b.iterrows():
                    price = row['PRICE']
                    dif = spectrum.best_bid - price
                    if dif > step*49:
                        pass
                    else:
                        new_bids[9 - distance_idx(dif, step)] += row['VOLUME']

                spectrum.bids = new_bids
            elif new_price == spectrum.best_bid:
                spectrum.bids[9] += row['VOLUME']
            else:
                dif = spectrum.best_bid - new_price
                if dif > step*49:
                    pass
                else:
                    spectrum.bids[9 - distance_idx(dif, step)] += row['VOLUME']

        if row['BUYSELL'] == 'S':
            if new_price < spectrum.best_ask:
                # temp_s = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'S')]
                temp_s = df_cur[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'S')]

                spectrum.best_ask = new_price
                new_asks = [0]*10

                for index, row in temp_s.iterrows():
                    price = row['PRICE']
                    dif = price - spectrum.best_ask
                    if dif > step*49:
                        pass
                    else:
                        new_asks[distance_idx(dif, step)] += row['VOLUME']

                spectrum.asks = new_asks
            elif new_price == spectrum.best_ask:
                spectrum.asks[0] += row['VOLUME']
            else:
                dif = new_price - spectrum.best_ask
                if dif > step*49:
                    pass
                else:
                    spectrum.asks[distance_idx(dif, step)] += row['VOLUME']

    # проверить если прайс = бест
    #elif row['ACTION'] == Action.REVOKE:
    else:
        if row['BUYSELL'] == 'B':

            if new_price == spectrum.best_bid:
                if row['VOLUME'] >= spectrum.bids[9]:
                    # temp_b = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'B')]
                    temp_b = df_cur[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'B')]
                    spectrum.best_bid = temp_b['PRICE'].max()
                    new_bids = [0]*10

                    for index, row in temp_b.iterrows():
                        price = row['PRICE']
                        dif = spectrum.best_bid - price
                        if dif > step*49:
                            pass
                        else:
                            new_bids[9 - distance_idx(dif, step)] += row['VOLUME']

                    spectrum.bids = new_bids
                else:
                    spectrum.bids[9] -= row['VOLUME']

            elif new_price > spectrum.best_bid:
                pass
            else:
                dif = spectrum.best_bid - new_price
                if dif > step*49:
                    pass
                else:
                    spectrum.bids[9 - distance_idx(dif, step)] -= row['VOLUME']

                if spectrum.bids[9 - distance_idx(dif, step)] < 0:
                    spectrum.bids[9 - distance_idx(dif, step)] = 0

        if row['BUYSELL'] == 'S':

            if new_price == spectrum.best_ask:
                if row['VOLUME'] >= spectrum.asks[0]:
                    # temp_s = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'S')]
                    temp_s = df_cur[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'S')]

                    spectrum.best_ask = temp_s['PRICE'].min()
                    new_asks = [0]*10

                    for index, row in temp_s.iterrows():
                        price = row['PRICE']
                        dif = price - spectrum.best_ask
                        if dif > step*49:
                            pass
                        else:
                            new_asks[distance_idx(dif, step)] += row['VOLUME']

                    spectrum.asks = new_asks
                else:
                    spectrum.asks[0] -= row['VOLUME']

            elif new_price < spectrum.best_ask:
                pass
            else:
                dif = new_price - spectrum.best_ask
                if dif > step*49:
                    pass
                else:
                    spectrum.asks[distance_idx(dif, step)] -= row['VOLUME']
                    if spectrum.asks[distance_idx(dif, step)] < 0:
                        spectrum.asks[distance_idx(dif, step)] = 0

    b = spectrum.bids.copy()
    b.extend(spectrum.asks)

    return b

In [96]:
start = time.time()
k = 0
# empty df for keeping track
df_cur = pd.DataFrame(columns=df_order.columns)
df_spec = pd.DataFrame(columns=['SECCODE', 'TIMESTAMP', 'BID_ASK'])

# for processing each TRADENO only once
matches = []

# iterating through the order book
# TODO: the loop to improve
for index, row in df_order.iterrows():
    # if post -> add to df_cur
    if row['ACTION'] == Action.POST:
        df_cur = df_cur.append(row)


    elif row['ACTION'] == Action.REVOKE:
        handle_revoke(df_cur, row)


    elif row['ACTION'] == Action.MATCH:
        if row['TRADENO'] in matches:
            matches.remove(row['TRADENO'])
            continue
        else:
            matches.append(row['TRADENO'])
            handle_match(df_cur, row)

    # для каждой новой row считаем спектрум и добавляем в файлик
    values = change_spectrums(df_cur, row)
    d = { 'SECCODE': row['SECCODE'], 'TIMESTAMP': row['TIME'], 'BID_ASK': values }
    df_spec = df_spec.append(d, ignore_index=True)
    #print(values)
    k += 1
    if k%50000 == 0:
        df_spec.to_csv('/content/drive/MyDrive/DataMining/spectrums/'+str(k//50000)+'.csv')


df_cur.to_csv('no-thread-lol.csv')
df_spec.to_csv('spectrum.csv')

end = time.time()

print(end - start)

----------------------------------------
in row: 13
ERROR: Cannot revoke more that there is
----------------------------------------

----------------------------------------
in row: 16
ERROR: Cannot revoke more that there is
----------------------------------------

----------------------------------------
in row: 19
ERROR: Cannot revoke more that there is
----------------------------------------

----------------------------------------
in row: 20
ERROR: Record with ORDERNO 16 doesn't exist
----------------------------------------



ValueError: cannot convert float NaN to integer

In [ ]:
df_spec.to_csv('/content/drive/MyDrive/DataMining/spectrum.csv')

In [ ]:
df_spec

In [ ]:
final = df_cur.groupby(['SECCODE','BUYSELL','PRICE'])['VOLUME'].sum().reset_index()\
    .sort_values(by=['SECCODE', 'BUYSELL', 'PRICE'])

In [ ]:
final

In [ ]:
final.to_csv('OrderBook' + OrderLog[8:] +'.csv')